In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix,classification_report,recall_score,confusion_matrix,mean_squared_error
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score,ShuffleSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing 
import re
from sklearn.preprocessing import LabelEncoder 
from pandas_profiling import ProfileReport
pd.set_option("display.precision", 6)
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('Jan_2019_ontime.csv')
df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
0,1,2,9E,20363,9E,N8688C,3280,11953,1195302,GNV,...,ATL,601.0,0.0,0600-0659,722.0,0.0,0.0,0.0,300.0,NaN
1,1,2,9E,20363,9E,N348PQ,3281,13487,1348702,MSP,...,CVG,1359.0,0.0,1400-1459,1633.0,0.0,0.0,0.0,596.0,NaN
2,1,2,9E,20363,9E,N8896A,3282,11433,1143302,DTW,...,CVG,1215.0,0.0,1200-1259,1329.0,0.0,0.0,0.0,229.0,NaN
3,1,2,9E,20363,9E,N8886A,3283,15249,1524906,TLH,...,ATL,1521.0,0.0,1500-1559,1625.0,0.0,0.0,0.0,223.0,NaN
4,1,2,9E,20363,9E,N8974C,3284,10397,1039707,ATL,...,FSM,1847.0,0.0,1900-1959,1940.0,0.0,0.0,0.0,579.0,NaN


In [ ]:
set_res = set(df.OP_UNIQUE_CARRIER)
set_res2 = set(df.ORIGIN)
set_res3 = set(df.DEST ) 

list_res = (list(set_res))
list_res2 = (list(set_res2))
list_res3 = (list(set_res2))
 
for item in list_res: 
    print(item) 

# for item2 in list_res2: 
#     print(item2) 

# for item3 in list_res3: 
#     print(item3) 



In [ ]:
def add_company(inpt):
    if inpt == 'ATL' or inpt == 'ORD':
        return inpt
    else:
        return 'Other'

In [ ]:
df['ORIGIN'] = df['ORIGIN'].apply(add_company)

In [ ]:
df['DEST'] = df['DEST'].apply(add_company)

In [ ]:
set_res2 = set(df.OP_UNIQUE_CARRIER)
list_res2 = (list(set_res2))

for item2 in list_res2: 
    print(item2) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [ ]:
df.head()

,DAY_OF_MONTH,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,DISTANCE,DELAYED
0,1,9E,Other,ATL,601.0,722.0,300.0,0
1,1,9E,Other,Other,1359.0,1633.0,596.0,0
2,1,9E,Other,Other,1215.0,1329.0,229.0,0
3,1,9E,Other,ATL,1521.0,1625.0,223.0,0
4,1,9E,ATL,Other,1847.0,1940.0,579.0,0


In [ ]:
#concatenate the arrival and departure delay 
df['DELAYED'] = (df['ARR_DEL15'].astype(bool) | df['DEP_DEL15'].astype(bool)).astype(int)

#decrease dimensions
df.drop(['OP_CARRIER_AIRLINE_ID','DEP_TIME_BLK','DAY_OF_WEEK','TAIL_NUM','OP_CARRIER_FL_NUM','ORIGIN_AIRPORT_ID',
         'ORIGIN_AIRPORT_SEQ_ID','DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID','Unnamed: 21','OP_CARRIER','ARR_DEL15','DEP_DEL15','CANCELLED', 'DIVERTED'], axis=1, inplace=True)

# check object columns
strings_columns =  list(df.dtypes[df.dtypes == 'object'].index)
print(f"The number of string columns is: {len(strings_columns)}")
    
# float columns
numeric_columns = list(df.drop(strings_columns,axis=1))
print(f"The number of numeric columns is: {len(numeric_columns)}")


The number of string columns is: 3
The number of numeric columns is: 5


In [ ]:
# to get percentage of null values on the whole data
(df.isna().sum() / df.shape[0]) * 100

DAY_OF_MONTH         0.000000
OP_UNIQUE_CARRIER    0.000000
ORIGIN               0.000000
DEST                 0.000000
DEP_TIME             2.800072
ARR_TIME             2.921479
DISTANCE             0.000000
DELAYED              0.000000
dtype: float64

In [ ]:
df.dropna(inplace=True)
df.isna().any()

df.shape

df.duplicated().sum()

df.drop_duplicates(inplace=True)
df.duplicated().sum()

df.hist(bins=100,figsize=(20,15))
plt.grid()
plt.show()

In [ ]:
# function to check outliers with box plot

def check_outliers(df):
    for col in numeric_columns:
        fig,ax= plt.subplots(figsize=(10,5))
        plt.grid()
        sns.boxplot(x = df[col])
        plt.show()

# Function to replace outliers with mean
def replace_outliers(df):
    for column in numeric_columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3-q1
        upper_lim = q3 + 1.5 * iqr
        lower_lim = q1 - 1.5 * iqr
        column_mean = df[column].mean()
        outliers_down = (df[column] < lower_lim)
        outliers_up = (df[column] > upper_lim)
        df[column] = np.where((df[column] > upper_lim) | (df[column] < lower_lim) , column_mean, df[column])
        df[column] = pd.DataFrame(df[column],columns=[column])
    return (df)

In [ ]:
#replace_outliers(df)

fig,ax = plt.subplots(figsize=(15,9))
sns.heatmap(df.corr(),annot=True,cmap="YlGnBu")
plt.show()

def encode_categories(features):
    lb_make = LabelEncoder()
    for i in range(len(features)):
        df[features[i]] = lb_make.fit_transform(df[features[i]])

# after preparing data, we have to make the final check before entering modeling phase
df.isna().sum()

df.duplicated().sum()

check_outliers(df)


In [ ]:
# The data is clean as there is no duplication, null values, and there is an outlier which is distance column but we can not handle it as the person may have a flight between 2 far countries, and it happens therefore the distance will be an outlie

df.corr()['DELAYED']

# As there is a correlation between the folloing encoded column and label so, we will encode them and drag them to the model
encode_categories(['OP_UNIQUE_CARRIER' , 'ORIGIN' , 'DEST'])

df.corr()['DELAYED']

df.head(607347)

df.info()

fig,ax = plt.subplots(figsize=(15,9))
sns.heatmap(df.corr(),annot=True,cmap="YlGnBu")
plt.show()

df.head(50)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 566922 entries, 0 to 583984
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DAY_OF_MONTH       566922 non-null  int64  
 1   OP_UNIQUE_CARRIER  566922 non-null  int64  
 2   ORIGIN             566922 non-null  int64  
 3   DEST               566922 non-null  int64  
 4   DEP_TIME           566922 non-null  float64
 5   ARR_TIME           566922 non-null  float64
 6   DISTANCE           566922 non-null  float64
 7   DELAYED            566922 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 38.9 MB


,DAY_OF_MONTH,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,DISTANCE,DELAYED
0,1,0,2,0,601.0,722.0,300.0,0
1,1,0,2,2,1359.0,1633.0,596.0,0
2,1,0,2,2,1215.0,1329.0,229.0,0
3,1,0,2,0,1521.0,1625.0,223.0,0
4,1,0,0,2,1847.0,1940.0,579.0,0
5,1,0,2,2,853.0,953.0,574.0,0
6,1,0,2,0,1553.0,1832.0,341.0,0
7,1,0,2,2,1551.0,1824.0,585.0,0
8,1,0,2,2,1037.0,1239.0,833.0,0
9,1,0,0,2,1245.0,1318.0,533.0,0


In [ ]:
X = df.drop('DELAYED',axis=1)
y = df['DELAYED']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)

models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

for name, model in models:
    
    print(name)
    
    # Make predictions.
    
    trained_model = model.fit(x_train, y_train)
        
    predictions = trained_model.predict(x_test) 
    
    print(f"train score: {accuracy_score(y_train, trained_model.predict(x_train))}\n")
    
    print(f"test score: {accuracy_score(predictions,y_test)}\n\n")

LogisticRegression
train score: 0.7781350401290176

test score: 0.7765129911745856


RandomForest
train score: 0.9999773211203367

test score: 0.8163890473138637


Decision Tree
train score: 0.9999974801244819

test score: 0.7455035072349583


KNN
train score: 0.8593103100706825

test score: 0.8104505606284212




In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
trained_model = rf.fit(x_train, y_train)
trained_model.fit(x_train, y_train)  
predictions = trained_model.predict(x_test) 
rf_old_train = accuracy_score(y_train, trained_model.predict(x_train))
pred = rf.predict(x_test)
rf_old_accur = accuracy_score(y_test, pred)

# new accuracy
rf = RandomForestClassifier(n_estimators=150, min_samples_split=5, max_features='sqrt', max_depth=20)
rf.fit(x_train,y_train)
trained_model = rf.fit(x_train, y_train)
trained_model.fit(x_train, y_train)  
predictions = trained_model.predict(x_test) 
rf_new_train = accuracy_score(y_train, trained_model.predict(x_train))
pred = rf.predict(x_test)
rf_new_accur = accuracy_score(y_test, pred)

print("The Training Accuracy of RandomForest Model before tuning: " + str(rf_old_train))
print("The Testing Accuracy of RandomForest Model before tuning: " + str(rf_old_accur))
print()
print("The Training Accuracy of RandomForest Model after tuning: " + str(rf_new_train))
print("The Testing Accuracy of RandomForest Model after tuning: " + str(rf_new_accur))
print()

The Training Accuracy of RandomForest Model before tuning: 0.9999773211203367
The Testing Accuracy of RandomForest Model before tuning: 0.8141959230231013

The Training Accuracy of RandomForest Model after tuning: 0.848101903765954
The Testing Accuracy of RandomForest Model after tuning: 0.8082809550968091



In [ ]:
x_train.columns

Index(['DAY_OF_MONTH', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME',
       'ARR_TIME', 'DISTANCE'],
      dtype='object')

In [ ]:
import pickle
with open('modelFDM.pickle', 'wb') as file:
    pickle.dump(trained_model, file)

In [ ]:
pred_value = trained_model.predict([[1,	0,	2,	0,	1550.0,	1859.0,	579.0	]])
pred_value

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


array([0])